In [ ]:
with open("/content/readers_books_clean.txt", 'r') as f:
    str1 = f.read()
    str1 = str1.replace("READER_ID:LibriVox_Book_IDs\n", '')


In [ ]:
with open("/content/readers_books_other.txt", 'r') as f:
    str2 = f.read()
    str2 = str2.replace("READER_ID:LibriVox_Book_IDs\n", '')


In [ ]:
book_dict = dict()
for line in str1.split('\n'):
    line = line.strip()
    if not line:
        continue
    i, books = line.split(':')
    books = books.split('|')
    for b in books:
        if b in book_dict:
            print(f"{b}!!!")
            book_dict[b].append(i)
        else:
            book_dict[b] = [i]
# book_dict

In [ ]:
for line in str2.split('\n'):
    line = line.strip()
    if not line:
        continue
    i, books = line.split(':')
    books = books.split('|')
    for b in books:
        if b in book_dict:
            print(f"{b}!!!")
            book_dict[b].append(i)
        else:
            book_dict[b] = [i]
# book_dict

In [41]:
import urllib3
!pip -qq install xmltodict
import xmltodict

http = urllib3.PoolManager()

import pandas as pd

In [48]:
df_book = pd.DataFrame(columns=[
    "book_id", "author_id", "reader_id",
    'title', 'language',
    'copyright_year',
    'num_sections',
    'totaltime', 'totaltimesecs',
    'url_librivox', 'url_project',
    "audio_type"
])

df_author = pd.DataFrame(columns=[
    'author_id', 
    'first_name', 'last_name', 
    'dob', 'dod'
])

In [ ]:
book_dict = dict()
for line in str1.split('\n'):
    line = line.strip()
    if not line:
        continue
    i, books = line.split(':')
    books = books.split('|')
    for b in books:
        if b in book_dict:
            raise KeyboardInterrupt
        else:
            book_dict[b] = i
for b, reader_id in book_dict.items():
    response_body = http.request('GET', f'https://librivox.org/api/feed/audiobooks/?id={b}')
    tree_dict = xmltodict.parse(response_body.data)
    if len(tree_dict["xml"]) != 1:
        raise KeyboardInterrupt("xml!")
    elif len(tree_dict["xml"]["books"]) != 1:
        raise KeyboardInterrupt("books!")

    temp_list = list()
    if type(tree_dict["xml"]["books"]["book"]["authors"]["author"]) is list:
        for a in tree_dict["xml"]["books"]["book"]["authors"]["author"]:
            author_row = dict()
            author_row["author_id"] = a["id"]
            temp_list.append(a["id"])
            author_row["first_name"] = a["first_name"]
            author_row["last_name"] = a["last_name"]
            author_row["dob"] = a["dob"]
            author_row["dod"] = a["dod"]

            df_author = df_author.append(pd.DataFrame([author_row]))
    else:
        author_row = dict()
        author_row["author_id"] = tree_dict["xml"]["books"]["book"]["authors"]["author"]["id"]
        temp_list.append(tree_dict["xml"]["books"]["book"]["authors"]["author"]["id"])
        author_row["first_name"] = tree_dict["xml"]["books"]["book"]["authors"]["author"]["first_name"]
        author_row["last_name"] = tree_dict["xml"]["books"]["book"]["authors"]["author"]["last_name"]
        author_row["dob"] = tree_dict["xml"]["books"]["book"]["authors"]["author"]["dob"]
        author_row["dod"] = tree_dict["xml"]["books"]["book"]["authors"]["author"]["dod"]

        df_author = df_author.append(pd.DataFrame([author_row]))
    
    book_row = dict()
    book_row["book_id"] = b
    book_row["author_id"] = sorted(temp_list)
    book_row["reader_id"] = reader_id
    book_row["title"] = tree_dict["xml"]["books"]["book"]["title"]
    book_row["language"] = tree_dict["xml"]["books"]["book"]["language"]
    book_row["copyright_year"] = tree_dict["xml"]["books"]["book"]["copyright_year"]
    book_row["num_sections"] = tree_dict["xml"]["books"]["book"]["num_sections"]
    book_row["totaltime"] = tree_dict["xml"]["books"]["book"]["totaltime"]
    book_row["totaltimesecs"] = tree_dict["xml"]["books"]["book"]["totaltimesecs"]
    book_row["url_librivox"] = tree_dict["xml"]["books"]["book"]["url_librivox"]
    book_row["url_project"] = tree_dict["xml"]["books"]["book"]["url_project"]
    book_row["audio_type"] = "clean"

    df_book = df_book.append(pd.DataFrame([book_row]))


In [ ]:
book_dict = dict()
for line in str2.split('\n'):
    line = line.strip()
    if not line:
        continue
    i, books = line.split(':')
    books = books.split('|')
    for b in books:
        if b in book_dict:
            raise KeyboardInterrupt
        else:
            book_dict[b] = i
for b, reader_id in book_dict.items():
    response_body = http.request('GET', f'https://librivox.org/api/feed/audiobooks/?id={b}')
    tree_dict = xmltodict.parse(response_body.data)
    if len(tree_dict["xml"]) != 1:
        raise KeyboardInterrupt("xml!")
    elif len(tree_dict["xml"]["books"]) != 1:
        raise KeyboardInterrupt("books!")

    temp_list = list()
    if type(tree_dict["xml"]["books"]["book"]["authors"]["author"]) is list:
        for a in tree_dict["xml"]["books"]["book"]["authors"]["author"]:
            author_row = dict()
            author_row["author_id"] = a["id"]
            temp_list.append(a["id"])
            author_row["first_name"] = a["first_name"]
            author_row["last_name"] = a["last_name"]
            author_row["dob"] = a["dob"]
            author_row["dod"] = a["dod"]

            df_author = df_author.append(pd.DataFrame([author_row]))
    else:
        author_row = dict()
        author_row["author_id"] = tree_dict["xml"]["books"]["book"]["authors"]["author"]["id"]
        temp_list.append(tree_dict["xml"]["books"]["book"]["authors"]["author"]["id"])
        author_row["first_name"] = tree_dict["xml"]["books"]["book"]["authors"]["author"]["first_name"]
        author_row["last_name"] = tree_dict["xml"]["books"]["book"]["authors"]["author"]["last_name"]
        author_row["dob"] = tree_dict["xml"]["books"]["book"]["authors"]["author"]["dob"]
        author_row["dod"] = tree_dict["xml"]["books"]["book"]["authors"]["author"]["dod"]

        df_author = df_author.append(pd.DataFrame([author_row]))
    
    book_row = dict()
    book_row["book_id"] = b
    book_row["author_id"] = sorted(temp_list)
    book_row["reader_id"] = reader_id
    book_row["title"] = tree_dict["xml"]["books"]["book"]["title"]
    book_row["language"] = tree_dict["xml"]["books"]["book"]["language"]
    book_row["copyright_year"] = tree_dict["xml"]["books"]["book"]["copyright_year"]
    book_row["num_sections"] = tree_dict["xml"]["books"]["book"]["num_sections"]
    book_row["totaltime"] = tree_dict["xml"]["books"]["book"]["totaltime"]
    book_row["totaltimesecs"] = tree_dict["xml"]["books"]["book"]["totaltimesecs"]
    book_row["url_librivox"] = tree_dict["xml"]["books"]["book"]["url_librivox"]
    book_row["url_project"] = tree_dict["xml"]["books"]["book"]["url_project"]
    book_row["audio_type"] = "others"

    df_book = df_book.append(pd.DataFrame([book_row]))


In [51]:
print(len(df_author))
df_author = df_author.drop_duplicates()
print(len(df_author))

90
67


In [53]:
print(len(df_book))
print(df_book.book_id.nunique())

77
77


In [54]:
df_book.head()

,book_id,author_id,reader_id,title,language,copyright_year,num_sections,totaltime,totaltimesecs,url_librivox,url_project,audio_type
0,13885,[431],9017,"d'Artagnan Romances, Vol 3, Part 1: The Vicomt...",English,1860,104,31:29:59,113399,https://librivox.org/the-dartagnan-romances-vo...,https://en.wikipedia.org/wiki/The_Vicomte_of_B...,clean
0,14261,[431],9017,"d'Artagnan Romances, Vol 3, Part 3: The Man in...",English,1910,91,30:06:01,108361,https://librivox.org/the-dartagnan-romances-vo...,None,clean
0,15752,[431],9017,"Marie Antoinette Romances, Vol 4: Taking the B...",English,1891,26,08:29:59,30599,https://librivox.org/antoinette-romances-vol-4...,None,clean
0,15482,[411],9017,Thus Spake Zarathustra: A Book for All and Non...,English,1909,83,16:24:19,59059,https://librivox.org/thus-spake-zarathustra-by...,None,clean
0,13130,[431],9017,"d'Artagnan Romances, Vol 1: The Three Musketee...",English,1844,69,02:34:47,9287,https://librivox.org/the-dartagnan-romances-vo...,None,clean


In [63]:
df_book["book_id"] = df_book.book_id.astype(int)
df_book["reader_id"] = df_book.reader_id.astype(int)
df_book = df_book.sort_values(["audio_type", "book_id", "reader_id"])
df_book.head()

,book_id,author_id,reader_id,title,language,copyright_year,num_sections,totaltime,totaltimesecs,url_librivox,url_project,audio_type
0,6973,[3290],92,Living Alone,English,1919,10,4:21:41,15701,https://librivox.org/living-alone-by-stella-be...,None,clean
0,7967,[10739],92,Wonderful Adventures of Mrs. Seacole in Many L...,English,1857,19,05:57:53,21473,https://librivox.org/wonderful-adventures-of-m...,None,clean
0,8992,[10682],6097,Present Picture of New South Wales,English,1811,5,04:26:13,15973,https://librivox.org/the-present-picture-of-ne...,None,clean
0,9288,[11],92,Scarlet Letter (version 2),English,1850,27,09:25:01,33901,https://librivox.org/the-scarlet-letter-by-nat...,https://en.wikipedia.org/wiki/The_Scarlet_Letter,clean
0,9575,[5066],6097,"Astral Plane: Its Scenery, Inhabitants and Phe...",English,1895,6,03:29:45,12585,https://librivox.org/the-astral-plane-its-scen...,None,clean


In [64]:
df_book["author_id"] = df_book["author_id"].apply(lambda some_list: [int(x) for x in some_list])

In [65]:
df_book.head()

,book_id,author_id,reader_id,title,language,copyright_year,num_sections,totaltime,totaltimesecs,url_librivox,url_project,audio_type
0,6973,[3290],92,Living Alone,English,1919,10,4:21:41,15701,https://librivox.org/living-alone-by-stella-be...,None,clean
0,7967,[10739],92,Wonderful Adventures of Mrs. Seacole in Many L...,English,1857,19,05:57:53,21473,https://librivox.org/wonderful-adventures-of-m...,None,clean
0,8992,[10682],6097,Present Picture of New South Wales,English,1811,5,04:26:13,15973,https://librivox.org/the-present-picture-of-ne...,None,clean
0,9288,[11],92,Scarlet Letter (version 2),English,1850,27,09:25:01,33901,https://librivox.org/the-scarlet-letter-by-nat...,https://en.wikipedia.org/wiki/The_Scarlet_Letter,clean
0,9575,[5066],6097,"Astral Plane: Its Scenery, Inhabitants and Phe...",English,1895,6,03:29:45,12585,https://librivox.org/the-astral-plane-its-scen...,None,clean


In [66]:
df_book.to_csv("book_info.csv", index=None, sep='|')

In [56]:
df_author.head()

,author_id,first_name,last_name,dob,dod
0,431,Alexandre,Dumas,1802,1870
0,411,Friedrich,Nietzsche,1844,1900
0,11757,Bernhard Eduard,Fernow,1851,1923
0,171,H. Rider,Haggard,1856,1925
0,15091,Edward Douglas,Fawcett,1866,1960


In [61]:
df_author["author_id"] = df_author.author_id.astype(int)
df_author = df_author.sort_values("author_id")
df_author.head()

,author_id,first_name,last_name,dob,dod
0,2,Ray,Cummings,1887,1957
0,3,Sewell Peaslee,Wright,1897,1970
0,7,Captain S. P.,Meek,1894,1972
0,8,Arthur J.,Burks,1898,1974
0,11,Nathaniel,Hawthorne,1804,1864


In [62]:
df_author.to_csv("author_info.csv", index=None, sep='|')